### Analysis of Molecular Dynamics Simulations

This notebook provides all the steps for the analysis of biomolecular dynamics simulation considering the simulation engine used was NAMD2 or NAMD3. 

### This step is required only for simulation with periodic boundary conditions

__In the simulation runs with periodic boundary conditions turned on, one need to wrap and unwrap the system before the analysis.__

__Wrap__ : When the atoms of the system are not all in one periodic image, but are distributed over various images, this function wraps all atoms into the chosen image. It is also possible to change between different representations of the unit cell (orthorhombic or triclinic).

__unwrap__ : When overlong bonds (that stretch the whole system) occur and compounds (residues, segments, chains or fragments) are broken in the course of a simulation trajectory because atoms are wrapped around the periodic boundaries, this function will remove large jumps of atoms between consecutive frames.

1. To wrap and wrap the molecules we will use __PBCTools plugin from VMD__:

  a) Open topology and trajectory file in VMD using following command from the terminal  
     `vmd <topology_file>  <list_of_trajectores_in_correct_order>`  
    Example: `vmd ../common/5X3Z_mod_ion.prmtop ../04_prod/04_5X3Z_mod_prod.dcd`  
  
  b) Open TkConsole _(Extensions --> Tk Console)_ from VMD and type following commands on TkConsole   
  
  c) Wrap the molecule   
  `pbc wrap -orthorhombic -centersel "protein or nucleic" -center com -compound residue -all`   
  For more information about options refer to [here](https://www.ks.uiuc.edu/Research/vmd/plugins/pbctools/).  
  
  d) Unwrap the protein-nucleic atoms   
  `pbc unwrap -sel "protein or nucleic"`  

  e) Save the trajctory in new trajectory file  
  `animate write dcd Analysis/04_prod_unwrap.dcd`  
  
  f) Exit  
  `quit`  


__If there are multiple runs/parts using extended simulation for the same simulation all the runs/parts of the simulation can be merged together using CATDCD tool__

### CATDCD  
1. Concatenate several trajectory files  
`catdcd -o <output_name.dcd> <list_of_trajectory_files.dcd>`
2. Concatenate several trajectory files and reduce the size  
 `catdcd -stride 10 -o <output_name.dcd> <list_of_trajectory_files.dcd>`

### VMD
1. Open all the parts of the trajectory  
`vmd <topology_file.prmtop> <list_of_trajectory_files.dcd>`

2. Save the merged trajectory   
`click on File --> Save coordinates --> (File type as dcd) & (selected atoms as all) --> Click on Save`

 __For analysis part, we can remove water and ion molecules from the trajectory and topology files__ 
We will use __CPPTRAJ__ for doing this. All the scripts to analyze using CPPTRAJ can be found [here](link). 
1. __strip_wat_ions_ptraj.in__  

This script does the following -
    - centers and image the trajectory using autoimage, 
    - removes the WAT, Na+, Cl- and H atoms from trajectory 
    - aligns the system on core (given residue range)  
  
2. __strip_topol_ptraj.in__  
This script removes the WAT, Na+, Cl- and H atoms from topology file.  
  

3. __rms_ptraj.in__  
This script aligns the whole system on core (given residue range) and computes following -  
 * __RMSF__
    * RMSF by @CA and assigns bfactor
    * RMSF by residue 
 * __RMSD__
    * Perresidue RMSD for nucleotides (given residue range)
    * RMSD for whole complex/system (aligned on core)
    * RMSD for protein (aligned on core)
    * RMSD for nucleic acid (aligned on core)
    * RMSD for nucleic acid (aligned on nucleic acid)
 
 
4. __split_traj_ptraj.in__  
This script splits/divide the trajectory into several parts based on the given range.  


5. __strip_atoms_ptraj.in__  
This script removes the specific atoms from the trajectory.

__Review how many frames are in a trajectory file__

`cpptraj -p topology-file.prmtop -y coordinate-file.dcd -tl`

Create plots from the data generated 